Download the data:

In [6]:
%%bash
mkdir -p /pipelines/data/committees/kns_documentcommitteesession
curl -k https://production.oknesset.org/pipelines/data/committees/kns_documentcommitteesession_dataservice/datapackage.json > /pipelines/data/committees/kns_documentcommitteesession_dataservice/datapackage.json
curl -k https://production.oknesset.org/pipelines/data/committees/kns_documentcommitteesession_dataservice/kns_documentcommitteesession_dataservice.csv > /pipelines/data/committees/kns_documentcommitteesession_dataservice/kns_documentcommitteesession_dataservice.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1815  100  1815    0     0  45375      0 --:--:-- --:--:-- --:--:-- 45375
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.8M  100 15.8M    0     0  24.8M      0 --:--:-- --:--:-- --:--:-- 24.8M


The pipeline takes a long time to run for all committee sessions

You should limit to running on a subset of sessions with cache by adding a filter step to kns_documentcommitteesession

```
  - run: filter
    cache: true
    parameters:
      in:
      - CommitteeSessionID: 2063122
      - CommitteeSessionID: 2063126
```

Start a tika server for parsing doc / docx files:

```
docker run --rm -p 9998:9998 logicalspark/docker-tikaserver
```

Check that the Tika server is accessible via the notebook environment:

In [2]:
%%bash
curl 172.17.0.1:9998 | tail

<li><b>POST</b> <i><a href="/translate/all/{translator}/{dest}">/translate/all/{translator}/{dest}</a></i><br />Class: org.apache.tika.server.resource.TranslateResource<br />Method: autoTranslate<br />Produces: text/plain</li>
<li><b>POST</b> <i><a href="/translate/all/{translator}/{src}/{dest}">/translate/all/{translator}/{src}/{dest}</a></i><br />Class: org.apache.tika.server.resource.TranslateResource<br />Method: translate<br />Produces: text/plain</li>
<li><b>PUT</b> <i><a href="/unpack/all{id:(/.*)?}">/unpack/all{id:(/.*)?}</a></i><br />Class: org.apache.tika.server.resource.UnpackerResource<br />Method: unpackAll<br />Produces: application/zip<br />Produces: application/x-tar</li>
<li><b>PUT</b> <i><a href="/unpack/{id:(/.*)?}">/unpack/{id:(/.*)?}</a></i><br />Class: org.apache.tika.server.resource.UnpackerResource<br />Method: unpack<br />Produces: application/zip<br />Produces: application/x-tar</li>
<li><b>GET</b> <i><a href="/version">/version</a></i><br />Class: org.apache.

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8432  100  8432    0     0   121k      0 --:--:-- --:--:-- --:--:--  121k


Run the pipelines, setting the tika server environment var with the correct value:

In [7]:
!{'cd /pipelines; TIKA_SERVER_ENDPOINT=http://172.17.0.1:9998 dpp run --verbose ./committees/kns_documentcommitteesession'}

[./committees/kns_documentcommitteesession:T_0] >>> INFO    :05c0f424 RUNNING ./committees/kns_documentcommitteesession
[./committees/kns_documentcommitteesession:T_0] >>> INFO    :05c0f424 Collecting dependencies
[./committees/kns_documentcommitteesession:T_0] >>> INFO    :05c0f424 Running async task
[./committees/kns_documentcommitteesession:T_0] >>> INFO    :05c0f424 Waiting for completion
[./committees/kns_documentcommitteesession:T_0] >>> INFO    :05c0f424 Async task starting
[./committees/kns_documentcommitteesession:T_0] >>> INFO    :05c0f424 Searching for existing caches
[./committees/kns_documentcommitteesession:T_0] >>> INFO    :05c0f424 Building process chain:
[./committees/kns_documentcommitteesession:T_0] >>> INFO    :- load_resource
[./committees/kns_documentcommitteesession:T_0] >>> INFO    :- knesset.rename_resource
[./committees/kns_documentcommitteesession:T_0] >>> INFO    :- filter
[./committees/kns_documentcommitteesession:T_0] >>> INFO    :- download_document_commi

## Inspect the output

In [8]:
from dataflows import Flow, load, printer
documentcommitteesessions = Flow(load('/pipelines/data/committees/kns_documentcommitteesession/datapackage.json')).results()[0][0]

In [10]:
session_id = 2072573
session = [session for session in documentcommitteesessions if session['CommitteeSessionID'] == session_id][0]
session
# session_files = {
#     'download': '/pipelines/data/committees/download_document_committee_session/'+session['download_filename'],
#     'text': '/pipelines/data/committees/meeting_protocols_text/'+session['text_parsed_filename'],
#     'text_hash': '/pipelines/data/committees/meeting_protocols_text/'+session['text_parsed_filename']+'.hash',
#     'parts': '/pipelines/data/committees/meeting_protocols_parts/'+session['parts_parsed_filename'],
#     'parts_hash': '/pipelines/data/committees/meeting_protocols_parts/'+session['parts_parsed_filename']+'.hash',
# }
# session_files

{'DocumentCommitteeSessionID': 437298,
 'CommitteeSessionID': 2072573,
 'GroupTypeID': 87,
 'GroupTypeDesc': 'חומר רקע',
 'ApplicationID': 4,
 'ApplicationDesc': 'PDF',
 'FilePath': 'https://fs.knesset.gov.il//20/Committees/20_cs_bg_501366.pdf',
 'LastUpdatedDate': datetime.datetime(2018, 6, 25, 13, 47, 50),
 'download_filename': None,
 'download_filesize': 0,
 'download_crc32c': None,
 'download_error': None,
 'text_protocol_extension': None,
 'text_parsed_filename': None,
 'text_filesize': 0,
 'text_crc32c': None,
 'text_error': None,
 'parts_protocol_extension': None,
 'parts_parsed_filename': None,
 'parts_filesize': 0,
 'parts_crc32c': None,
 'parts_error': None}

In [25]:
import subprocess

for k, v in session_files.items():
    print(subprocess.check_output('ls -lah '+v, shell=True).decode().strip())

-rw-r--r--    1 root     root       46.0K Oct 15 06:17 /pipelines/data/committees/download_document_committee_session/files/23/4/3/434231.DOC
-rw-r--r--    1 root     root       83.1K Oct 15 18:02 /pipelines/data/committees/meeting_protocols_text/files/2/0/2063122.txt
-rw-r--r--    1 root     root          32 Oct 15 18:02 /pipelines/data/committees/meeting_protocols_text/files/2/0/2063122.txt.hash
-rw-r--r--    1 root     root       83.1K Oct 15 18:02 /pipelines/data/committees/meeting_protocols_parts/files/2/0/2063122.csv
-rw-r--r--    1 root     root          32 Oct 15 18:02 /pipelines/data/committees/meeting_protocols_parts/files/2/0/2063122.csv.hash
